# Projekt 3 - Eksperymenty i analiza danych
## Autorzy
- Daniel Zdancewicz [145317]
- Alex Terentowicz [145419]

## Opis
t 3 polega na przeprowadzeniu eksperymentów z użyciem kilku metod i przeprowadzeniu
analizy uzyskanych wyników. Znajdź zbiór danych, który zawiera kilka kryteriów monotonicznych, ponad 100 alternatyw i co najmniej 2 klasy (jeśli problem ma więcej klas, możesz je zbinaryzować).

Wyobraźmy sobie, że jesteśmy instytucją zajmującą się utylizacją odpadów radioaktywnych. Naszym
zadaniem jest zaproponowanie modelu utylizacji takich odpadów dla 30-letniego programu nuklearnego.
Wybrano 3 miejsca na potencjalne składowisko, 3 scenariusze czasowe oraz 3 modele finansowania. Ze
względu na to, że ostatecznie klient może się nie zgodzić na optymalną dla nas strategię, dlatego chcielibyśmy uzyskać pełne rankingi wszystkich opcji. Zespół ekspertów ocenił wszystkie możliwe scenariusze przez nas rozważane (dane znajdują się w pliku [Nuclear waste management.csv](.


Krótko opisz zbiór danych, w tym opisy kryteriów. Dla tego zbioru danych wytrenuj następujące modele:
- Jeden prosty, interpretowalny model ML (np. XGBoost, rankSVM lub regresja logistyczna)
- Jeden interpretowalny model ANN-MCDA przedstawiony na zajęciach (ANN-Ch-Constr.
lub ANN-UTADIS)
- Złożony model sici neuronowej zawierającej kilka warstw ukrytych.


## Opis zbioru danych

### Opisy kryteriów


## Trening modeli

### Rank SVM — interpretowalny model ML

### ANN-Ch-Constr. — interpretowalny model ANN-MCDA

### Głęboka sieć neuronowa — model złożony.

## Eksperymenty
Wszystkie prezentowane wartości są do maksymalnie 4 miejsc po przecinku.

### Przedstawienie metryk

- Rank SVM
    - Accuracy
    - F1
    - AUC
- ANN-Ch-Constr.
    - Accuracy
    - F1
    - AUC
- Głęboka sieć neuronowa
    - Accuracy
    - F1
    - AUC


### Przedstawienie modelu
(np funkcje cząstkowe, schemat drzewa ...).


### Interpretacja
- Czy na podstawie uzyskanych parametrów możemy powiedzieć coś o preferencjach
użytkowników?
- Czy są jakieś kryteria, które nie mają żadnego wpływu, czy też mają wpływ decydujący.
- Czy istnieją jakieś progi preferencji?
- Czy istnieją oceny kryteriów, które są nierozróżnialne z punktu widzenia preferencji?


### Interpretacja techniką Partial Dependence Plot
Wykonaj interpretację modelu korzystając z conajmniej jednej techniki( Global Surrogate,
Partial Dependence Plot Permutation Feature Importance ...)


Dla 3 wybranych wariantów dokonaj wyjaśnienia decyzji.
- Znajdź minimalną zmianę w ocenach tak aby wariant został przypisany do innej klasy.
    – Spróbuj odpowiedzieć na to pytanie w sposób analityczny, opierając się tylko na
wartościach parametrów modelu i wyjaśnij, dlaczego taka zmiana jest minimalna
(bez próbkowania).
    - Wykonaj próbkowanie przestrzeni, nieznacznie zmieniając oceny, tak aby uzyskać
inną klasę. Czy wyniki zgadzają się z przewidywaniami teoretycznymi?
- Dokonaj wyjaśnienia predykcji za pomocą conajmniej jednej techniki (Anchors LIME,
SHAP, ... )


##### Możliwe narzędzia
Lista narzędzi, które zawierają różne techniki wyjaśniania predykcji i interpretacji modelu:
- Shapash
- Alibi
- Explainerdashboard
- DALEX
- eli5
- aix360